In [40]:
import pinecone
import os
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [41]:
load_dotenv()

True

In [42]:
api_key = os.getenv('PINECONE_API_KEY')

In [43]:
os.environ["PINECONE_API_KEY"] = api_key

In [44]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [31]:
loader = PyPDFLoader("pdf/Ronald_E_Riggio_Introduction_to_Industrial_Orga.pdf")
data = loader.load()

In [9]:
#Página 139 - 194
data[139:194]

[Document(page_content='The Performance Appraisal Process\nLegal Concerns in Performance Appraisals\nTeam Appraisals\nSummary\nInside Tips\nEMPLOYEE PERFORMANCE: A CRITERION FOR SUCCESS\nThis chapter looks at how employees’ job performance is measured and appraised in organizations. Often,\nmeasures of performance are the criteria used to determine the effectiveness of an employee testing or screening\nprogram as discussed in the previous chapter. Because job performance is such an important outcome variable\nin I/O psychology, it is important to understand the measurement issues concerning this factor. For example,\nwhen reviewing studies that discuss influences on job performance, you should investigate how performance\nwas operationally defined and measured. Were objective or subjective criteria used? How accurate or inaccurate\nmight the assessments of performance be? How can performance assessments and appraisals be improved?\nJob Performance and Performance Appraisals\nFrom the f

In [10]:
#Página 223 - 278
data[223:278]

[Document(page_content='You are getting settled into your new job. A great deal of effort went into finding the position, making it\nthrough the screening process, and landing the job and in your initial training and orientation. You’ve learned\nthe ropes and know what to do. There are a lot of positive aspects to the job, but it’s not all rosy. Your attitudes\nand evaluation of your working situation will determine if you stay in this job, with this company, and even\non this career path. We work partly out of necessity, but we stay in a job or an organization because of the\npositive things that come from the job, the company, and the career.\nSeventy-five years ago, the only compensation that most workers received from their jobs was a paycheck.\nAs time went on, this changed as workers began to demand and receive more from their jobs. Today’s workers\nreceive a variety of forms of compensation, including health care, retirement, and numerous other benefits and\nprograms. However, o

In [11]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 400, 
    chunk_overlap = 20
)

#Filtrando solo las páginas importantes para el proyecto
docs = text_splitter.split_documents(documents = data[139:194] + data[223:278])

In [12]:
len(docs)

110

In [45]:
index_name = os.getenv('INDEX_NAME')

pc = pinecone.Pinecone(api_key = api_key)
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'bdproyecto-9spvjnj.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'bdproyecto',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [14]:
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)

d:\00 Clases\DMC Peru\06_IA Generativa con LLM\Proyecto_Final\proyecto-final-env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [46]:
index = pc.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0011,
 'namespaces': {'': {'vector_count': 110}},
 'total_vector_count': 110}


In [47]:
embed = OpenAIEmbeddings()
text_field = "text"

vectorstore = Pinecone(
    index, 
    embed.embed_query, 
    text_field
)

d:\00 Clases\DMC Peru\06_IA Generativa con LLM\Proyecto_Final\proyecto-final-env\lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [78]:
template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé, formula bien tu pregunta."

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

prompt = PromptTemplate(
    template = template, input_variables = ["context", "question"]
)

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever(),
    verbose = True,
    chain_type_kwargs = {"prompt": prompt, "verbose": True}
)

In [73]:
qa.run("Qué preguntas realizarías para detectar el estrés en empleados?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé, formula bien tu pregunta."

Contexto: 
Self-Efficacy
 an individual’s beliefs in his or her abilities to engage in courses of action that will lead to
desired outcomes
Measurement of Worker Stress
Because stress is such a complex phenomenon and because stress researchers cannot agree on a single
definition of stress, you might suspect that the measurement of stress is extremely difficult. For the most part,
measurement of stress in general, and of worker stress in particular, is problematic. There have been a number
of approaches to measuring stress. We will consider several of these.
Physiological Measures
As has been stated, the stress response involves physiological reactions as well as psychological and emotion

'Algunas preguntas que podrían realizarse para detectar el estrés en empleados podrían ser:\n1. ¿Cuántos proyectos/asignaciones tienes actualmente?\n2. ¿Cuánto tiempo pasas en reuniones?\n3. ¿Cuánto tiempo pasas en el trabajo?\n4. ¿Cuántas llamadas telefónicas y visitantes tienes durante el día?\n5. ¿Cómo te sientes en general en el trabajo?\n6. ¿Qué aspectos de tu trabajo te generan más estrés?\n7. ¿Cómo te sientes física y emocionalmente en el trabajo?\n8. ¿Qué cambios en el ambiente laboral podrían ayudarte a reducir el estrés?\n9. ¿Qué estrategias utilizas para lidiar con el estrés en el trabajo?\n10. ¿Cómo crees que el estrés afecta tu desempeño laboral y tu bienestar en general?'

In [75]:
qa.run("realiza 5 preguntas para detectar el estrés en empleados")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé, formula bien tu pregunta."

Contexto: 
readings on facts and myths about stress, the impact of stress on physical and psychological
health and on work performance, and potential sources of stress.
2
. 
Self-analysis: Learning about your personal stress (3 hours)
—This section can include
assessments of personal stressors using instruments such as the stressful life events scale or
workers’ self-reports.
3
. 
Methods of coping with work stress (3 hours)
—Here various individual coping strategies are
presented and perhaps demonstrated.
4
. 
Developing a personalized coping plan (3 hours)
—In this final part participants work on
developing customized programs for managing stress, including setting personal stress
mana

'1. ¿Cuántos proyectos o tareas tienes actualmente?\n2. ¿Cuánto tiempo pasas en reuniones durante el día?\n3. ¿Cuánto tiempo pasas en el trabajo?\n4. ¿Cuántas llamadas telefónicas y visitas recibes durante el día?\n5. ¿Cómo te sientes en general en tu ambiente laboral?'

In [76]:
qa.run("realiza 5 preguntas para una evaluación de desempeño laboral")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé, formula bien tu pregunta."

Contexto: 
of Chicago
, 1978). Moreover, performance appraisals need to be administered and scored under controlled and
standardized conditions (
Brito v. Zia Company
, 1973). Specifically, court cases have ruled that appraisers must
receive training, or at least written instructions, on how to conduct performance appraisals, that assessments
must focus on performance-related behaviors rather than on performance-related personality traits or other
dispositional variables, and that appraisals must be reviewed with the employees (Barrett & Kernan, 1987; Feild
& Holley, 1982). In addition, just as the Americans with Disabilities Act (ADA) requires that employers make
reasonable accommodatio

'1. ¿Cuáles son los criterios de evaluación de desempeño utilizados en tu trabajo?\n2. ¿Cómo te evaluarías a ti mismo en términos de competencias y comportamientos de equipo?\n3. ¿Qué aspectos de tu desempeño laboral crees que podrían mejorar?\n4. ¿Cómo crees que la evaluación de desempeño afecta tu desarrollo profesional?\n5. ¿Qué sugerencias tienes para mejorar el proceso de evaluación de desempeño en nuestra organización?'

In [81]:
qa.run("realiza 5 preguntas para saber la satisfacción laboral del empleado")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé, formula bien tu pregunta."

Contexto: 
Stop 
&
 Review
List and define five categories of organizational citizenship behaviors.
Summary
Job satisfaction
, which involves the positive feelings and attitudes one has about a job, can be conceptualized
in overall, or global, terms or in terms of specific components or facets and can be measured through
interviews or with self-report instruments. The most widely used self-report measures are the 
Minnesota
Satisfaction Questionnaire
 (
MSQ
) and the 
Job Descriptive Index
 (
JDI
). Research indicates that there is a slight
positive relationship between job satisfaction and job performance, although the link may be moderated by
another variable, such as the receipt of wo

'1. ¿Cómo calificaría su nivel de satisfacción con su trabajo actual?\n2. ¿Qué aspectos específicos de su trabajo le generan mayor satisfacción?\n3. ¿Qué cambios o mejoras le gustaría ver en su trabajo para aumentar su satisfacción?\n4. ¿Cómo describiría su relación con sus compañeros de trabajo y supervisores en términos de satisfacción?\n5. ¿Qué actividades o tareas adicionales realiza en su trabajo que contribuyen a su satisfacción laboral?'